# Manejo con pipelines

En este tutorial aprenderemos a trabajar con pipelines y lo mezclaremos con estrategias de optimización de parámetros.

Usaremos el dataset del Titanic donde nos encontraremos con los siguientes temas:
   - Habrá variables nulas
   - Tendremos variables categóricas y numéricas
   - El dataset se encuentra desbalanceado


In [ ]:
%matplotlib inline
#######
# Importamos todas las dependencias
import numpy as np
import matplotlib.pyplot as plt


from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder, OrdinalEncoder
#from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn import metrics

np.random.seed(42)

X, y = fetch_openml("titanic", version=1, as_frame=True, return_X_y=True)
X.head(10)

In [ ]:
y.value_counts()

Como vemos este dataset se encuentra algo desbalanceado. Lo tendremos que tener en cuenta en los siguientes momentos:
   - Al dividir en muestras de aprendizaje y test
   - Cuando usemos Cross Validation
   - En el entrenamiento de los modelos. Casi todos los métodos en sklearn admiten un parámetro `class_weight='balanced'`

In [ ]:
# Borramos columnas que no nos interesan
X.drop(['boat', 'body', 'home.dest', 'ticket', 'name'], axis=1, inplace=True)
# Seleccionados train/test en modo stratify, ya que el dataset se encuentra desbalanceado
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)

In [ ]:
X.describe()

## Tratamiento de valores nulos

In [ ]:
X_train.isnull().any()

In [ ]:
X_train.isnull().sum()

Vamos a dibujar los valores que son nulos

In [ ]:
import missingno as msno
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'png'

msno.matrix(X_train)
plt.show()

In [ ]:
X_train.isnull().sum() / len(X_train) * 100

El atributo 'cabin' tiene demasiados valores nulos. En este caso no tiene sentido aplicar ningún método de imputación, por lo que vamos a eliminar

In [ ]:
X_train.drop(['cabin'], axis=1, inplace=True)
X_test.drop(['cabin'], axis=1, inplace=True)

Vamos a calcular las correlaciones de los valores numéricos

In [ ]:
import pandas as pd
import seaborn as sns
%config InlineBackend.figure_format = 'svg'

X_comb = pd.concat([X_train, y_train.astype(float)], axis=1)
g = sns.heatmap(X_comb[['pclass', 'age', 'sibsp', 'parch', 'fare', 'survived']].corr(),
                annot=True,
                cmap = "coolwarm")

In [ ]:
# Si fuera tipo objeto evaluar "OrdinalEncoder"
X['pclass'].value_counts()

## Codificamos la salida
Aunque la variable de salida es 0 y 1, pasamos a codificarla con `LabelEncoder` ya que para algunas funciones como el cálculo de las curvas ROC necesitamos esta codificación.

In [ ]:
# Es necesario
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y)

y_test = le.transform(y_test)
y_train = le.transform (y_train)

## Analizar tipos de variables
Vamos a analizar las variables categóricas que tenemos en el dataset

In [ ]:
X_train.dtypes

In [ ]:
cat_cols = X_train.select_dtypes(include="category").columns
num_cols = X_train.select_dtypes(exclude="category").columns

In [ ]:
cat_cols

Para las variables categóticas vamos a crear un pipeline donde:
1. Si hubiese una muestra con valores nulos, le vamos a imputar el valor del más frecuente.
2. Codificaremos con la técnicas One_Hot_Encoder.
3. Aplicaremos sobre ellos un PCA, para extraer características más discriminantes

In [ ]:
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False)),
    ('pca', PCA(n_components=5))
])

In [ ]:
# Si queremos acceder al primer paso del pipeline. De la misma forma podríamos acceder a los distintos pasos del pipeline
cat_transformer[0]

Ahora nos metemos con las variables numéricas. En este caso vamos a realizar el siguiente preproceso:
1. Si hay valores nulos, vamos a asignarles la media de sus K=5 vecinos más cercanos.
2. Realizaremos un escalado

In [ ]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import RobustScaler

num_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=5)),
    ('scaler', RobustScaler())
])

En el siguiente código vamos a utilizar la función `ColumnTransformer` para indicar cómo vamos a preprocesar a las variables categóricas y numéricas

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_cols),
        ('cat', cat_transformer, cat_cols)
    ])


In [ ]:
preprocessor


## Creamos el clasificador
Por último, vamos a crear un `Pipeline` para unir el paso del preproceso, con el paso de entrenamiento del modelo. Fijaros que en el entrenamiento del modelo estoy indicando `class_weight='balanced'`, ya que el dataset se encuentra desbalanceado.

In [ ]:
from sklearn.model_selection import cross_val_score
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', tree.DecisionTreeClassifier(class_weight='balanced'))])

Si el valor del parámetro es `cv` asignado a un número entero y la variable de salida es binaria o multiclase, entonces utiliza `StratifiedKFold`. Directamente utiliza el valor de `shuffle=False`.
En este caso, lleva a cabo un `StratifiedKFold` con K=5, y obtenemos el valor medio del `accuracy`.

In [ ]:
clf

Como el dataset está desbalanceado usamos como mejor resultado "balanced_accuracy"

In [ ]:
cross_val_score(clf, X_train, y_train, cv=5, scoring="balanced_accuracy").mean()

Ahora vamos a buscar los mejores hiperparámetros

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

num_transformer_dist = {'preprocessor__num__imputer__n_neighbors': list(range(2, 15)),
                        'preprocessor__num__imputer__add_indicator': [True, False]}

cat_transformer_dist = {'preprocessor__cat__imputer__strategy': ['most_frequent', 'constant'],
                        'preprocessor__cat__imputer__add_indicator': [True, False],
                        'preprocessor__cat__pca__n_components': list(range(2, 5))}

random_forest_dist = {'classifier__max_depth': list(range(2, 20)),
                      'classifier__min_samples_split': list(range(20, 200))}

param_dist = {**num_transformer_dist, **cat_transformer_dist, **random_forest_dist}

random_search = RandomizedSearchCV(clf,
                                   param_distributions=param_dist,
                                   n_iter=100)

In [ ]:
random_search.fit(X_train, y_train)

In [ ]:
random_search.best_score_

In [ ]:
random_search.best_params_

In [ ]:
y_pred = random_search.predict(X_test)
y_pred[:10]

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

La precisión balanceada se usa en problemas de clasificación binaria y multiclase para tratar conjuntos de datos desbalanceados. Se define como la media del recall obtenido en cada clase.

In [ ]:
from sklearn.metrics import balanced_accuracy_score

print(f"El valor de balanced accuracy: {balanced_accuracy_score(y_test, y_pred)}")


In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

cm = confusion_matrix(y_test, y_pred, labels=random_search.classes_)
disp= ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=random_search.classes_)
disp.plot()
plt.show()

In [ ]:
#Obtenemos las curva ROC y el área bajo la curva (AUC)

probs = random_search.predict_proba(X_test)[:, 1]

auc = metrics.roc_auc_score(y_test, probs)
fpr, tpr, thresholds = metrics.roc_curve(y_test, probs)

plt.figure(figsize=(8, 5))
plt.plot(fpr, tpr, label=f'AUC  = {auc:.2f}')
plt.plot([0, 1], [0, 1], color='blue', linestyle='--', label='Baseline')
plt.title('Curva ROC', size=20)
plt.xlabel('Falsos Positivos', size=14)
plt.ylabel('Verdaderos Positivos', size=14)
plt.legend();

In [ ]:
# Modelo final
modelo_final = random_search.best_estimator_
_ = modelo_final.fit (X,y)
